In [722]:
import sys
import os
from subprocess import Popen, PIPE
import re

In [723]:
PRAAT= '/usr/bin/praat'
#PRAAT = 'D:\Downloads\praat6104_win64\Praat.exe'

diphone_folder = "/home/aletg/ph-tp1/difonos/"
#diphone_folder = "D:\Documents\Facu\ph-tp1\difonos\\"

In [724]:
def run_praat(script_file, args):
    args= [PRAAT, '--run', script_file, args]
    p = Popen(args, stdout=PIPE)
    p.wait()

Lista de archivos de audio - concatenar audio y escribir a archivo

In [725]:
def output_audio(filenames, output_name):

    script = ""
    for i, f in enumerate(filenames):
        script += "Read from file: \"" + f + "\"\n"
        script += "selectObject: \"Sound " + f[-6:-4] + "\"\n"
        script += "Rename: \"" + f[-6:-4] + str(i) + "\"\n"

    script += "selectObject: \"Sound " + filenames[0][-6:-4] + str(0) + "\"\n"
    for i, f in enumerate(filenames[1:]):
        script += "plusObject: \"Sound " + f[-6:-4] + str(i+1) + "\"\n"

    script += "Concatenate recoverably\n"
    script += "selectObject: \"Sound chain\"\n"
    script += "Write to WAV file: \"" + output_name + "\"\n"
    script += "Save as text file: \"" + output_name + ".TextGrid\"\n"

    with open("concatenar.praat", "w") as cf:
        cf.write(script)

    run_praat("concatenar.praat", "")
    os.remove("concatenar.praat")

# output_audio(["./Difonos/re.wav", "./Difonos/rE.wav"], "./file.wav")

Obtener lista de difonos a partir de una palabra

In [726]:
diphones = ["-e", "-E", "-f", "-k", "-m", "-s", "-t", "e-", "E-",
           "s-", "ee", "eE", "Ee", "EE", "fe", "fE", "ke", "kE", 
           "me", "mE", "se", "sE", "te", "tE", "re", "rE", "ef",
           "Ef", "ek", "Ek", "em", "Em", "es", "Es", "et", "Et",
           "er", "Er", "fr", "kr", "tr", "sf", "sk", "st", "ss",
           "sm"]

wavsufix = ".wav"

def diphone_list(word):
    diphone_list = []
    word  = "-" + word + "-"
    for i, _ in enumerate(word[:-1]):
        diphone = word[i:i+2]
        if not diphone in diphones:
            print("ERROR: invalid word. Try again.")
            sys.exit(1)
        diphone_list.append(diphone)
    print(diphone_list)
    # agregar prefijo y sufijo
    return list(map(lambda s: diphone_folder + s + wavsufix, diphone_list))

diphone_list("efe")

['-e', 'ef', 'fe', 'e-']


['/home/aletg/ph-tp1/difonos/-e.wav',
 '/home/aletg/ph-tp1/difonos/ef.wav',
 '/home/aletg/ph-tp1/difonos/fe.wav',
 '/home/aletg/ph-tp1/difonos/e-.wav']

In [843]:
pitch_start_range = 90
pitch_end_range = 300

def extract_pitch_track(filename):
    args = " "
    args += filename + ".wav " + filename + ".PitchTier " + str(pitch_start_range) + " " + str(pitch_end_range)
    run_praat("extraer-pitch-track.praat", args )

def replace_pitch_track(inputSound, inputPitch, outSound):
    args = " "
    args += inputSound + " " + inputPitch + " " + outSound + " " + str(pitch_start_range) + " " + str(pitch_end_range)
    run_praat("reemplazar-pitch-track.praat", args )

In [844]:
def modify_pitch_track( pitchTierIn, factor, pitchTierOut ):
    with open(pitchTierIn, 'r') as inF, open(pitchTierOut, "w") as outF:
        for line in inF:
            if "value" in line:
                pitch = float(line[12:])
                pitch *= factor
                line = "    value = " + str(pitch) + "\n"
            outF.write(line)

In [845]:
modify_pitch_track( "./test/estemeketrefe.PitchTier", 2, "./test/estemeketrefe_agudo.PitchTier" )


In [846]:
def tts(word, output_filename):
    output_audio(diphone_list(word), output_filename)

In [847]:
def test(word):
    tts(word, "./test/" + word + ".wav")
    
def test_dbg(list, name):
    output_audio(list, "./test/" + name + ".wav")
    

Ejemplo de ejecución del TTS

In [854]:
def variar_pitch(inF, factor):
    extract_pitch_track( inF )

    modify_pitch_track( inF + ".PitchTier", factor, inF + "_agudo.PitchTier" )

    replace_pitch_track( inF + ".wav", inF + "_agudo.PitchTier", inF + "_agudo.wav" )

In [861]:
variar_pitch("./difonos/fe", 2)

test("estemeketrefe")
variar_pitch("./test/estemeketrefe", 2)

['-e', 'es', 'st', 'te', 'em', 'me', 'ek', 'ke', 'et', 'tr', 're', 'ef', 'fe', 'e-']


In [853]:
variar_pitch("./s",2 )

In [135]:
test("fefefEfe")

['-f', 'fe', 'ef', 'fe', 'ef', 'fE', 'Ef', 'fe', 'e-']


In [143]:
test("fefefe")

['-f', 'fe', 'ef', 'fe', 'ef', 'fe', 'e-']
['D:\\Downloads\\praat6104_win64\\Praat.exe', '--run', 'concatenar.praat', '']


In [87]:
test("sefefe")

['-s', 'se', 'ef', 'fe', 'ef', 'fe', 'e-']


In [88]:
test("sefEfefese")

['-s', 'se', 'ef', 'fE', 'Ef', 'fe', 'ef', 'fe', 'es', 'se', 'e-']


In [78]:
test("sefefes")

['-s', 'se', 'ef', 'fe', 'ef', 'fe', 'es', 's-']


In [79]:
test("efefe")

['-e', 'ef', 'fe', 'ef', 'fe', 'e-']


In [89]:
test("mes")
test_dbg(diphone_list("mes")[1:], "mesa")

['-m', 'me', 'es', 's-']
['-m', 'me', 'es', 's-']


In [60]:
test("es")

['-e', 'es', 's-']


In [61]:
test("mefes")

['-m', 'me', 'ef', 'fe', 'es', 's-']


In [62]:
test("mese")

['-m', 'me', 'es', 'se', 'e-']


In [90]:
test("meme")
test_dbg(diphone_list("eme")[1:], "eme!")

['-m', 'me', 'em', 'me', 'e-']
['-e', 'em', 'me', 'e-']


In [91]:
test("emes")

['-e', 'em', 'me', 'es', 's-']


In [65]:
test("eme")

['-e', 'em', 'me', 'e-']


In [66]:
test("ke")

['-k', 'ke', 'e-']


In [67]:
test("keke")

['-k', 'ke', 'ek', 'ke', 'e-']


In [92]:
test("ekefeke")

['-e', 'ek', 'ke', 'ef', 'fe', 'ek', 'ke', 'e-']


In [93]:
test("esefeseke")

['-e', 'es', 'se', 'ef', 'fe', 'es', 'se', 'ek', 'ke', 'e-']


In [546]:
test("kese")

['-k', 'ke', 'es', 'se', 'e-']


In [547]:
test("keses")

['-k', 'ke', 'es', 'se', 'es', 's-']


In [94]:
test("kekeke")

['-k', 'ke', 'ek', 'ke', 'ek', 'ke', 'e-']


In [549]:
test("te")

['-t', 'te', 'e-']


In [550]:
test("tese")

['-t', 'te', 'es', 'se', 'e-']


In [551]:
test("teketeke")

['-t', 'te', 'ek', 'ke', 'et', 'te', 'ek', 'ke', 'e-']


In [552]:
test("tetete")

['-t', 'te', 'et', 'te', 'et', 'te', 'e-']


In [553]:
test("fetre")

['-f', 'fe', 'et', 'tr', 're', 'e-']


In [95]:
test("meketrefe")
test("esemeketrefe")

['-m', 'me', 'ek', 'ke', 'et', 'tr', 're', 'ef', 'fe', 'e-']
['-e', 'es', 'se', 'em', 'me', 'ek', 'ke', 'et', 'tr', 're', 'ef', 'fe', 'e-']


In [555]:
test("ererere")
test("eskereskere")

['-e', 'er', 're', 'er', 're', 'er', 're', 'e-']
['-e', 'es', 'sk', 'ke', 'er', 're', 'es', 'sk', 'ke', 'er', 're', 'e-']


In [556]:
test("este")

['-e', 'es', 'st', 'te', 'e-']


In [139]:
test("estemeketrefe")
test("estemekestefe")

['-e', 'es', 'st', 'te', 'em', 'me', 'ek', 'ke', 'et', 'tr', 're', 'ef', 'fe', 'e-']
['D:\\Downloads\\praat6104_win64\\Praat.exe', '--run', 'concatenar.praat', '']
['-e', 'es', 'st', 'te', 'em', 'me', 'ek', 'ke', 'es', 'st', 'te', 'ef', 'fe', 'e-']
['D:\\Downloads\\praat6104_win64\\Praat.exe', '--run', 'concatenar.praat', '']


In [558]:
test("efre")
test("fre")

['-e', 'ef', 'fr', 're', 'e-']
['-f', 'fr', 're', 'e-']


In [559]:
test("fretre")

['-f', 'fr', 're', 'et', 'tr', 're', 'e-']


In [96]:
test("freskete")
test("kefreskete")
test("kekefreskete")

['-f', 'fr', 're', 'es', 'sk', 'ke', 'et', 'te', 'e-']
['-k', 'ke', 'ef', 'fr', 're', 'es', 'sk', 'ke', 'et', 'te', 'e-']
['-k', 'ke', 'ek', 'ke', 'ef', 'fr', 're', 'es', 'sk', 'ke', 'et', 'te', 'e-']


In [561]:
test("kekEke")

['-k', 'ke', 'ek', 'kE', 'Ek', 'ke', 'e-']


In [97]:
test("kekefreskEke")

['-k', 'ke', 'ek', 'ke', 'ef', 'fr', 're', 'es', 'sk', 'kE', 'Ek', 'ke', 'e-']


In [563]:
test("kefreskEte")
test("kEfreskEte")

['-k', 'ke', 'ef', 'fr', 're', 'es', 'sk', 'kE', 'Et', 'te', 'e-']
['-k', 'kE', 'Ef', 'fr', 're', 'es', 'sk', 'kE', 'Et', 'te', 'e-']


In [564]:
test("ekre")
test("tekre")
test("kekrete")

['-e', 'ek', 'kr', 're', 'e-']
['-t', 'te', 'ek', 'kr', 're', 'e-']
['-k', 'ke', 'ek', 'kr', 're', 'et', 'te', 'e-']


In [98]:
test("esfe")
test("esfesfesfe")
test("esfesfesfesfesfesfesfe")

['-e', 'es', 'sf', 'fe', 'e-']
['-e', 'es', 'sf', 'fe', 'es', 'sf', 'fe', 'es', 'sf', 'fe', 'e-']
['-e', 'es', 'sf', 'fe', 'es', 'sf', 'fe', 'es', 'sf', 'fe', 'es', 'sf', 'fe', 'es', 'sf', 'fe', 'es', 'sf', 'fe', 'es', 'sf', 'fe', 'e-']


In [566]:
test_dbg(diphone_list("esfes")[1:], "esfe!")

['-e', 'es', 'sf', 'fe', 'es', 's-']


In [567]:
test("este")
test("esste")

['-e', 'es', 'st', 'te', 'e-']
['-e', 'es', 'ss', 'st', 'te', 'e-']


In [568]:
test_dbg(diphone_list("esme")[2:], "esme!")
test("esme")
test("esmesmesme")

['-e', 'es', 'sm', 'me', 'e-']
['-e', 'es', 'sm', 'me', 'e-']
['-e', 'es', 'sm', 'me', 'es', 'sm', 'me', 'es', 'sm', 'me', 'e-']


In [569]:
test("Efe")

['-E', 'Ef', 'fe', 'e-']


In [570]:
test("E")
test("EfE")
test("EfefefEfefE")

['-E', 'E-']
['-E', 'Ef', 'fE', 'E-']
['-E', 'Ef', 'fe', 'ef', 'fe', 'ef', 'fE', 'Ef', 'fe', 'ef', 'fE', 'E-']


In [571]:
test("eEke")

['-e', 'eE', 'Ek', 'ke', 'e-']


In [572]:
test("Eeke")
test("Eefe")

['-E', 'Ee', 'ek', 'ke', 'e-']
['-E', 'Ee', 'ef', 'fe', 'e-']


In [573]:
test("efe")

['-e', 'ef', 'fe', 'e-']


In [99]:
test("EEEEEEEeeeEE")
test("eeeeeeeee")

['-E', 'EE', 'EE', 'EE', 'EE', 'EE', 'EE', 'Ee', 'ee', 'ee', 'eE', 'EE', 'E-']
['-e', 'ee', 'ee', 'ee', 'ee', 'ee', 'ee', 'ee', 'ee', 'e-']


In [575]:
test("fEEfEE")

['-f', 'fE', 'EE', 'Ef', 'fE', 'EE', 'E-']


In [576]:
test("mEfe")
test("mEfemE")

['-m', 'mE', 'Ef', 'fe', 'e-']
['-m', 'mE', 'Ef', 'fe', 'em', 'mE', 'E-']


In [577]:
test("sE")
test("sEfE")
test("sEke")

['-s', 'sE', 'E-']
['-s', 'sE', 'Ef', 'fE', 'E-']
['-s', 'sE', 'Ek', 'ke', 'e-']


In [578]:
test("tE")
test("tEfE")
test("tEkE")
test("etEkE")

['-t', 'tE', 'E-']
['-t', 'tE', 'Ef', 'fE', 'E-']
['-t', 'tE', 'Ek', 'kE', 'E-']
['-e', 'et', 'tE', 'Ek', 'kE', 'E-']


In [579]:
test("erE")
test("erEfE")
test("erEkE")
test("erEkE")

['-e', 'er', 'rE', 'E-']
['-e', 'er', 'rE', 'Ef', 'fE', 'E-']
['-e', 'er', 'rE', 'Ek', 'kE', 'E-']
['-e', 'er', 'rE', 'Ek', 'kE', 'E-']


In [580]:
test("erEme")
test("erEmE")
test("ereme")
test("erEmefe")

['-e', 'er', 'rE', 'Em', 'me', 'e-']
['-e', 'er', 'rE', 'Em', 'mE', 'E-']
['-e', 'er', 're', 'em', 'me', 'e-']
['-e', 'er', 'rE', 'Em', 'me', 'ef', 'fe', 'e-']


In [581]:
test("erEse")
test("erEsE")
test("erese")
test("erEsefe")

['-e', 'er', 'rE', 'Es', 'se', 'e-']
['-e', 'er', 'rE', 'Es', 'sE', 'E-']
['-e', 'er', 're', 'es', 'se', 'e-']
['-e', 'er', 'rE', 'Es', 'se', 'ef', 'fe', 'e-']


In [100]:
test("erEre")
test("erErE")
test("erere")
test("erErefe")

['-e', 'er', 'rE', 'Er', 're', 'e-']
['-e', 'er', 'rE', 'Er', 'rE', 'E-']
['-e', 'er', 're', 'er', 're', 'e-']
['-e', 'er', 'rE', 'Er', 're', 'ef', 'fe', 'e-']
